In [1]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI



In [2]:
# 1. PDF 문서 로딩
loader = PyPDFLoader("docs/sample.pdf")
pages = loader.load()
print(f"Loaded {len(pages)} pages from the PDF document.")
print(f"pages: {pages} \n")  # Print the loaded pages for debugging
print(f"First page content: {pages[0].page_content[:200]}...")  # Print first 200 characters of the first page
print(f"pages: {pages[0].page_content} \n")  # Print the loaded pages for debugging

# 2. 페이지를 청크로 분할  --  맥락 고려해서 청크 사이즈 조정하기. (근거가 뭐임?? - 문서를 내가 정규화 시켜볼까 고민)
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
print(f"text_splitter: {text_splitter} \n")  # Print the text splitter configuration for debugging
docs = text_splitter.split_documents(pages)
print(f"Split into {len(docs)} chunks.")
print(f"docs: {docs} \n")  # Print the split documents for debugging
print(f"First chunk content: {docs[0].page_content[:200]}...")  # Print first 200 characters of the first chunk


Loaded 4 pages from the PDF document.
pages: [Document(metadata={'producer': 'Hancom PDF 1.3.0.550', 'creator': 'Hwp 2018 10.0.0.14515', 'creationdate': '2025-06-16T21:51:53+09:00', 'author': 'USER', 'moddate': '2025-06-16T21:51:53+09:00', 'pdfversion': '1.4', 'source': 'docs/sample.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='핵심 역량 / Strength   - Python을 활용한 역산 및 최적화 프로그램 개발 경험 중앙대학교 도로교통 연구실에서 Python을 활용하여 활주로 최적화 및 도로 균열 예측 시스템을 개발하였습니다. 데이터 수집, 전처리, 시각화 및 알고리즘 구현 등 전체 파이프라인에 참여하여 실질적인 분석 및 개선 성과를 이끌었습니다.   - 임베디드 시스템을 활용한 빌딩 환경 측정 플랫폼 구축 경험Esp32와 C를 기반으로 한 IoT 디바이스를 구축하고, 수집된 데이터를 MySQL 및 Grafana로 시각화하는 전          주기 개발 경험을 보유하고 있습니다.   - 모바일 프론트엔드 개발 및 팀 리딩 경험 (React Native)KAIST App 창업 프로그램에서 건설 유지보수 관련 앱을 개발하며 프론트엔드 파트를 리딩하였습니다. React Native를 활용한 UI 구성, API 연동, 사용자 편의성 개선에 주력하여 실제 서비스를 배포하였습니다.    - 글로벌 협업 기반의 웹 개발 경험캐나다 Tamwood Web-Developer Co-op 과정 중 다양한 국적의 개발자들과 팀을 이루어 소셜미디어, 미니게임 등 웹 기반 토이 프로젝트를 React와 Node.js를 활용해 개발하며 영어 소통 및 협업 역량을 키웠습니다.학력 사항 / 

In [3]:
# 3. 벡터화 + FAISS 저장소 생성.   임베딩 모델 라지모델 더 많이 씀.!! - 청크 사이즈가 커서 맥락 이해 더 잘함!
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_key="DaH9GW5nK15zX9b1541B4H7lt2l9YeXLm6xzWzrdhjlZw322zTINJQQJ99BGACYeBjFXJ3w3AAABACOGbelF",
    openai_api_version="2023-05-15", 
    azure_endpoint="https://rounders-openai.openai.azure.com",  
    chunk_size=1_000,
)

vectorstore = FAISS.from_documents(docs, embeddings)


/var/folders/wy/_jj74ny11pd8y1qwn7xj5vc80000gn/T/ipykernel_10238/3201574562.py:2: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embeddings = AzureOpenAIEmbeddings(


In [54]:
print(vectorstore)
for i, (doc_id, doc) in enumerate(vectorstore.docstore._dict.items()):
    print(f"[Document {i+1}]")
    print(f"내용: {doc.page_content[:300]}...\n")  # 처음 300자만 출력

print(f"벡터 개수: {vectorstore.index.ntotal}")


[Document 1]
내용: 핵심 역량 / Strength   - Python을 활용한 역산 및 최적화 프로그램 개발 경험 중앙대학교 도로교통 연구실에서 Python을 활용하여 활주로 최적화 및 도로 균열 예측 시스템을 개발하였습니다. 데이터 수집, 전처리, 시각화 및 알고리즘 구현 등 전체 파이프라인에 참여하여 실질적인 분석 및 개선 성과를 이끌었습니다.   - 임베디드 시스템을 활용한 빌딩 환경 측정 플랫폼 구축 경험Esp32와 C를 기반으로 한 IoT 디바이스를 구축하고, 수집된 데이터를 MySQL 및 Grafana로 시각화하는 전          주기 ...

[Document 2]
내용: 교육 사항 / Education Experience교육기관명교육기간교육내용캐나다(Tamwood)Web-Developer Co-op 과정2023.05 ~ 2023.10HTML, CSS, JavaScript, Git 등 웹 개발 기초부터 React를 활용한 프로젝트까지 수료 KAIST App 창업프로그램 4기2024.11 ~ 2025.02Figma, React, Spring 기반 서비스 기획 및 MVP 개발창업 실무 및 기술 워크숍을 병행한 실습 위주의 교육 과정MICROSOFT AI 엔지니어 과정2025.05 ~ 2025.08최신 A...

[Document 3]
내용: 자기소개서<직무 성장과정 ( 자기소개 )>어떤 특정 기술만 잘하는 개발자가 아닌, 스스로 기획하고, 문제를 정의하고, 직접 구현해나가는 진짜 ‘개발자’가 되고 싶었습니다. 저는 컴퓨터 시스템을 통해 세상을 더 편리하게 만들 수 있다는 점에서 매력을 느껴, AI 개발자와 SI/웹앱 개발자를 주요 진로로 설정하게 되었습니다.처음에는 단순히 점수에 맞춰 컴퓨터 관련 학과에 진학했지만, 타 대학원 연구실에서 프로그램을 개발하며 누군가의 삶을 편리하게 만든다는 기쁨을 처음 느꼈습니다. 그 경험은 단순한 ‘코딩’이 아닌, 실질적인 ‘창조’의 ...

[Document 4]
내용: 위의 기재한 내용은 사실과 다름

In [ ]:
llm = AzureChatOpenAI(
    deployment_name="o4-mini",  
    openai_api_key="DaH9GW5nK15zX9b1541B4H7lt2l9YeXLm6xzWzrdhjlZw322zTINJQQJ99BGACYeBjFXJ3w3AAABACOGbelF",
    api_version="2024-12-01-preview",
    azure_endpoint="https://rounders-openai.openai.azure.com/",
    temperature=1 ,
)
print("LLM initialized successfully.", llm)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",  # or "map_reduce", "refine" if 문서 많을 때
)




LLM initialized successfully. client=<openai.resources.chat.completions.completions.Completions object at 0x126b56e00> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1289e1c60> temperature=1.0 model_kwargs={} openai_api_key='DaH9GW5nK15zX9b1541B4H7lt2l9YeXLm6xzWzrdhjlZw322zTINJQQJ99BGACYeBjFXJ3w3AAABACOGbelF' openai_proxy='' azure_endpoint='https://rounders-openai.openai.azure.com/' deployment_name='o4-mini' openai_api_version='2024-12-01-preview' openai_api_type='azure'
📌 답변: 아래는 지금까지 수행하신 주요 활동을 항목별로 정리한 내용입니다.  

1. 학력  
 - 성공회대학교 소프트웨어공학·정보통신공학 복수전공(2017.03 ~ 2025.08 예정, 평점 3.46/4.5)  

2. 연구실 및 공공 R&D 프로젝트  
 - 활주로 최적화 시스템 개발  
   · Python 기반 유전 알고리즘·하모니 서치 적용  
   · 비행기 종류별 수명 예측, 시뮬레이션 및 시각화  
   · MATLAB 알고리즘을 Python으로 이식·재설계  
 - 도로 균열 예측 시스템  
   · Python을 이용한 데이터 수집·전처리·모델링·시각화  
 - ESP32 기반 빌딩 환경 측정 플랫폼 구축  
   · 온습도·CO₂ 센서 데이터 수집 및 정규화  
   · MySQL 저장, Grafana 시각화  
 - 점토 블록 성능 실증용 데이터 수집·분석 도구 개발 (Python)  

3. 해외 연수 및 창업 프로그램  
 

In [ ]:
query = "내가 지금까지 한 활동들을 현대 오토에버에 맞는 자소서로 작성해줘"
answer = qa.run(query)
print("📌 답변:", answer)

📌 답변: [자기소개서]

1. 지원 동기  
현대오토에버는 커넥티드카, 스마트팩토리, 모빌리티 플랫폼 등 자동차 산업의 디지털 전환을 선도하며 미래 모빌리티 혁신을 이끄는 기업입니다. 저는 대학 시절부터 AI와 IoT·웹 개발을 융합하여 실사용자에게 가치를 제공하는 솔루션을 직접 기획·구현해 왔습니다. 특히 스마트 센서 플랫폼과 최적화 알고리즘 개발 경험을 토대로, 커넥티드카 데이터 분석 및 차량 내·외부 통합 서비스 구축에 기여하고 싶습니다. 현대오토에버의 ‘안전·편리·지속가능’ 모빌리티 비전에 동참하여, 사용자 경험을 극대화하는 스마트 모빌리티 서비스를 함께 발전시키고자 지원합니다.

2. 성장 과정 및 핵심 역량  
- 문제 정의부터 구현·분석·개선까지 전 과정 주도  
  중앙대학교 산학협력단 도로교통 연구실에서 ‘활주로 최적화 시스템’과 ‘도로 균열 예측 도구’를 Python 기반으로 개발하며, 도메인 이해→데이터 수집·전처리→유전 알고리즘·하모니 서치 최적화→시각화까지 전주기를 경험했습니다. 기존 MATLAB 코드를 구조적으로 재설계하고, 정책 제안용 대시보드를 제작하며 ‘기술이 현실 문제를 해결하는 과정’을 체득했습니다.

- 임베디드 IoT 플랫폼 설계·구축  
  ESP32와 온습도·CO₂ 센서를 이용한 빌딩 환경 측정 시스템을 처음부터 설계·구현했습니다. 센서 데이터의 표준화 및 노이즈 제거, MySQL 저장, Grafana 시각화까지 전 과정을 혼자 담당하며, 실시간 모니터링 요구사항을 충족시켰습니다. 이 경험을 통해 임베디드 개발, 데이터 파이프라인 구축 역량을 쌓았고, 커넥티드카 환경에서의 센서 데이터 수집·분석에도 유연하게 적용할 수 있습니다.

- 풀스택 웹·모바일 서비스 개발  
  캐나다 Tamwood Co-op 과정과 KAIST App 창업 프로그램 4기에서 React·React Native로 UI/UX를 설계하고, Node.js API 연동, AAB 배포까지 담당했습니다. 특히 건설 유지보수 앱 프로젝트에서는 프론트엔드 파트